In [ ]:
# import library
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import TfidfVectorizer
from sklearn.svm import SVC

In [ ]:
# read data from CSV file
df = pd.read_csv('../data/raw/marvelRanarokExcel.csv')
# df = pd.read_csv('marvelRanarok.csv', encoding="cp1252")

print(df.to_string())
print(df.loc[2, "review"])
print(df.info())
print(df["sentiment"].value_counts())
# print(pd.options.display.max_rows)

# convert label from text to digit
df["sentiment"] = df["sentiment"].map({"positive" : 1, "negative" : -1, "neutral" : 0})
X = df["review"] # input
y = df["sentiment"] # label


In [ ]:
# split train date and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# vectorize comments from text to vector
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
# train model
model = SVC(kernel="linear")
model.fit(X_train, y_train)

In [ ]:
# predict result